In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import xlrd
import plotly.express as px
import plotly.graph_objects as go
import warnings

In [35]:
#shape files
warnings.filterwarnings('ignore', 'GeoSeries.notna', UserWarning)
wsh = gpd.read_file('lv_subbasins61/lv_subbasins61.geojson') #basins
lvb = gpd.read_file('eLVB_withBuffer\eLVB_withBuffer.shp')
lakes = gpd.read_file('water_bodies\Africa_waterbody_fixed.shp') #waterbodies
lakes.to_crs(lvb.crs)
lakes = gpd.clip(lakes, wsh)
schemes = gpd.read_file('irrigation_schemes/irrigation_schemes_rice.geojson')
discharge = gpd.read_file('discharge\wfas_EA_discharge.shp')
data = pd.read_excel('output.xlsx')
prec = pd.read_excel('3_effective_precipitation.xls', sheet_name='output')

In [36]:
wsh = gpd.read_file('lv_subbasins61/lv_subbasins61.geojson')
wsh= wsh.set_index('gridcode2')

In [37]:
wsh1 = gpd.overlay(wsh, lakes, how='difference')

In [40]:
data['yield_difference'] = data['irr_int_g0_production_VS_S'].div(data['irr_int_g0_extent_VS_S'].replace(0,np.nan)).fillna(0) - data['rain_low_g0_production_VS_S'].div(data['rain_low_g0_extent_VS_S'].replace(0,np.nan)).fillna(0)
data['area_difference'] = data['irr_int_g0_extent_VS_S'] - data['rain_low_g0_extent_VS_S']
data['yield_irrigated'] = data['irr_int_g0_production_VS_S'].div(data['irr_int_g0_extent_VS_S'].replace(0,np.nan)).fillna(0)
data['increase_production'] = (data['rain_low_g0_extent_VS_S']*data['yield_difference']) + (data['area_difference']*data['yield_irrigated'])
data['gains'] = data['increase_production'] / data['irr_int_g0_extent_VS_S'] *10
data['water_avail'] = data['Dis_year_3'] - data['annual']
data['discharge'] =  (data['water_avail']/data['irr_int_g0_extent_VS_S']) / (data['gains']/data['irr_int_g0_extent_VS_S'])
data['production_capita'] = (data['irr_int_g0_production_VS_S'] - data['rain_low_g0_production_VS_S']) / data['pop_total_2030'] * 10000
data['pop_density'] = data['pop_total_2030'] / data['wsh_extent_total']

## Analysis

### Increase in production

G = $A_{r}(Y_{ir-r}) + (A_{ir}-A_{r})(Y_{ir})$


where $A =$ area, $Y=$ yields, $ir=$ irrigated + intermediate inputs, $r=$ rain fed + low inputs

In [41]:
data['gains'].describe()

count    59.000000
mean      2.615359
std       0.380117
min       1.663244
25%       2.407185
50%       2.587751
75%       2.818038
max       3.899618
Name: gains, dtype: float64

In [45]:
fig = px.choropleth_mapbox(data, geojson=wsh1, locations='gridcode', color='gains',
                           color_continuous_scale='Oryel',
                           zoom=4, center = {'lat': 0., 'lon': 32.4166},
                           labels={'gains':'tons / ha'},
                           hover_data=['wsh_code'], range_color=(2.2, 2.8), # opacity=0.1, 
                           title='Promising areas for increasing wetland rice production <br> in the extended Lake Victoria Basin')

fig.update_layout(autosize=False,
                  mapbox_style='carto-positron',
                  mapbox_layers=[{'sourcetype':'geojson', 
                                  'source':'irrigation_schemes/irrigation_schemes_rice.geojson', 
                                  'type':'fill',
                                  'color':'rgb(5,112,176)'}],
                  margin={"r":0,"t":100,"l":0,"b":0})



fig.show()

### Increase in production / water available for irrigation

In [43]:
data['discharge'].describe()

count    59.000000
mean      1.773431
std       3.311305
min       0.014108
25%       0.104282
50%       0.336666
75%       0.918423
max      11.738715
Name: discharge, dtype: float64

In [44]:
fig = px.choropleth_mapbox(data, geojson=wsh1, locations='gridcode', color='discharge',
                           color_continuous_scale='Earth',
                           mapbox_style="carto-positron",
                           zoom=4, center = {"lat": 0., "lon": 32.4166},
                           labels={'discharge':'km3 / ton'},
                           hover_data=['wsh_code'], range_color=(0.014108, 1),
                           title= 'Water available to increase production <br> with irrigation')

fig.update_layout(autosize=False,
                  mapbox_style='carto-positron',
                  margin={"r":0,"t":100,"l":0,"b":0})

fig.show()